# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])
import re
import os
import pickle
import numpy as np
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sqlalchemy import create_engine
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV, train_test_split

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [2]:
# load data from database
engine = create_engine('sqlite:///disaster_response.db')
df = pd.read_sql_table('disaster_response', engine)
X = df['message']
Y = df.drop(['message', 'id', 'original', 'genre'], axis=1)

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text,url_place_holder_string="urlplaceholder"):
    """
    Tokenize the text function
    
    Arguments:
        text -> Text message which needs to be tokenized
    Output:
        clean_tokens -> List of tokens extracted from the provided text
    """
    
    # Replace all urls with a urlplaceholder string
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    
    # Extract all the urls from the provided text 
    detected_urls = re.findall(url_regex, text)
    
    # Replace url with a url placeholder string
    for detected_url in detected_urls:
        text = text.replace(detected_url, url_place_holder_string)

    # Extract the word tokens from the provided text
    tokens = nltk.word_tokenize(text)
    
    #Lemmanitizer to remove inflectional and derivationally related forms of a word
    lemmatizer = nltk.WordNetLemmatizer()

    # List of clean tokens
    clean_tokens = [lemmatizer.lemmatize(w).lower().strip() for w in tokens]
    return clean_tokens

In [5]:
# Build a custom transformer which will extract the starting verb of a sentence
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        # tokenize by sentences
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            # tokenize each sentence into words and tag part of speech
            pos_tags = nltk.pos_tag(tokenize(sentence))
             # index pos_tags to get the first word and part of speech tag
            first_word, first_tag = pos_tags[0]
            # return true if the first word is an appropriate verb or RT for retweet
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, Y=None):
        return self

    def transform(self, X):
        # apply starting_verb function to all values in X
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
#Build the pipeline
pipeline = Pipeline([
       ('features', FeatureUnion([

           ('text_pipeline', Pipeline([
               ('vect', CountVectorizer(tokenizer=tokenize)),
               ('tfidf', TfidfTransformer())
            ]))
        ])),

        ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split the data
X_train, X_test, Y_train, Y_test = train_test_split(X, Y)

# train RandomForestClassifier
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [8]:
def test_report(model, X_test, Y_test):
    
    Y_pred = model.predict(X_test)
    k = 0
    for col in Y_test:
        print('Feature {}: {}'.format(k+1, col))
        print(classification_report(Y_test[col], Y_pred[:, k]))
        k = k + 1
    accuracy = (Y_pred == Y_test.values).mean()
    print('The model accuracy is {:.3f}'.format(accuracy))

In [9]:
test_report(pipeline, X_test, Y_test)

Feature 1: related
             precision    recall  f1-score   support

          0       0.57      0.19      0.29      1513
          1       0.79      0.96      0.87      4996
          2       0.12      0.02      0.04        45

avg / total       0.74      0.77      0.73      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.92      0.97      0.95      5454
          1       0.81      0.58      0.68      1100

avg / total       0.90      0.91      0.90      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3816
          1       0.77      0.61      0.68      2738

avg / total       0.76      0.76      0.75      6554

Feature 5

### 6. Improve your model
Use grid search to find better parameters. 

In [10]:
# Show parameters for the pipline
pipeline.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip...y=None)), ('tfidf', TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True))]))],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        

In [11]:
# Using grid search
# Create Grid search parameters for RandomForestClassifier 
parameters = {
     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
     'features__text_pipeline__tfidf__use_idf': (True, False),
}

cv = GridSearchCV(pipeline, param_grid=parameters, n_jobs=-1, cv=2, verbose=3)
cv

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [12]:
# Fit the RandomForestClassifier tuned model
cv.fit(X_train, Y_train)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.23781914352558234, total= 1.5min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1) 


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.7min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 1), score=0.19469026548672566, total= 1.5min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  3.3min remaining:    0.0s


[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.23232631471874682, total= 3.4min
[CV] features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  features__text_pipeline__tfidf__use_idf=True, features__text_pipeline__vect__ngram_range=(1, 2), score=0.20659139456820264, total= 3.3min
[CV] features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 1), score=0.23181771945885465, total= 1.4min
[CV] features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 1) 
[CV]  features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 1), score=0.20872749465974977, total= 1.3min
[CV] features__text_pipeline__tfidf__use_idf=False, features__text_pipeline__vect__ngram_range=(1, 2) 
[CV]  featu

[Parallel(n_jobs=-1)]: Done   8 out of   8 | elapsed: 20.4min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)), 'features__text_pipeline__tfidf__use_idf': (True, False)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [13]:
# Prediction: RandomForestClassifier 
test_report(cv, X_test, Y_test)

Feature 1: related
             precision    recall  f1-score   support

          0       0.57      0.22      0.32      1513
          1       0.80      0.95      0.87      4996
          2       0.52      0.24      0.33        45

avg / total       0.74      0.78      0.74      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.92      0.97      0.94      5454
          1       0.79      0.56      0.65      1100

avg / total       0.89      0.90      0.89      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3816
          1       0.77      0.61      0.68      2738

avg / total       0.76      0.76      0.75      6554

Feature 5

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

## Improve
Add  StartingVerbExtractor feature.

In [23]:
pipeline2 = Pipeline([
   ('features', FeatureUnion([

       ('text_pipeline', Pipeline([
           ('vect', CountVectorizer(tokenizer=tokenize)),
           ('tfidf', TfidfTransformer())
       ])),

       ('starting_verb', StartingVerbExtractor())
   ])),

   ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

In [24]:
# Fit the RandomForestClassifier
pipeline2.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))])

In [25]:
# Prediction
test_report(pipeline2, X_test, Y_test)

Feature 1: related
             precision    recall  f1-score   support

          0       0.57      0.19      0.29      1513
          1       0.79      0.96      0.87      4996
          2       0.12      0.02      0.04        45

avg / total       0.74      0.77      0.73      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.92      0.97      0.95      5454
          1       0.81      0.58      0.68      1100

avg / total       0.90      0.91      0.90      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.87      0.81      3816
          1       0.77      0.61      0.68      2738

avg / total       0.76      0.76      0.75      6554

Feature 5

In [26]:
# Show parameters for the pipline
pipeline2.get_params()

{'memory': None, 'steps': [('features', FeatureUnion(n_jobs=1,
          transformer_list=[('text_pipeline', Pipeline(memory=None,
        steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_... smooth_idf=True, sublinear_tf=False, use_idf=True))])), ('starting_verb', StartingVerbExtractor())],
          transformer_weights=None)),
  ('clf',
   MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None,
             learning_rate=1.0, n_estimators=50, random_state=None),
              n_jobs=1))], 'features': FeatureUnion(n_jobs=1,
        transformer_list=[('text_pipeline', Pipeline(memory=None,
      steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
         dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
    

In [29]:
# Using grid search
# Create Grid search parameters for RandomForestClassifier 
parameters = {
     'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2)),
}

cv2 = GridSearchCV(pipeline2, param_grid=parameters, n_jobs=-1, cv=2, verbose=3)
cv2

GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [30]:
# Fit the  tuned model
cv2.fit(X_train, Y_train)

Fitting 2 folds for each of 2 candidates, totalling 4 fits
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 1), score=0.23822601973349608, total= 2.3min
[CV] features__text_pipeline__vect__ngram_range=(1, 1) ...............


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  2.8min remaining:    0.0s


[CV]  features__text_pipeline__vect__ngram_range=(1, 1), score=0.19469026548672566, total= 2.3min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:  5.6min remaining:    0.0s


[CV]  features__text_pipeline__vect__ngram_range=(1, 2), score=0.2325297528227037, total= 4.6min
[CV] features__text_pipeline__vect__ngram_range=(1, 2) ...............
[CV]  features__text_pipeline__vect__ngram_range=(1, 2), score=0.20648967551622419, total= 4.5min


[Parallel(n_jobs=-1)]: Done   4 out of   4 | elapsed: 16.0min finished


GridSearchCV(cv=2, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('features', FeatureUnion(n_jobs=1,
       transformer_list=[('text_pipeline', Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_d...mator=None,
          learning_rate=1.0, n_estimators=50, random_state=None),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=-1,
       param_grid={'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=3)

In [31]:
# Prediction
test_report(cv2, X_test, Y_test)

Feature 1: related
             precision    recall  f1-score   support

          0       0.60      0.16      0.25      1513
          1       0.79      0.97      0.87      4996
          2       0.60      0.13      0.22        45

avg / total       0.74      0.78      0.72      6554

Feature 2: request
             precision    recall  f1-score   support

          0       0.92      0.97      0.94      5454
          1       0.78      0.60      0.68      1100

avg / total       0.90      0.90      0.90      6554

Feature 3: offer
             precision    recall  f1-score   support

          0       1.00      1.00      1.00      6523
          1       0.00      0.00      0.00        31

avg / total       0.99      0.99      0.99      6554

Feature 4: aid_related
             precision    recall  f1-score   support

          0       0.76      0.88      0.81      3816
          1       0.78      0.61      0.69      2738

avg / total       0.77      0.77      0.76      6554

Feature 5

### 9. Export your model as a pickle file

In [32]:
# Create a pickle file for the model
file_name = 'classifier.pkl'
with open (file_name, 'wb') as f:
    pickle.dump(cv, f)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.